In [2]:
import sys
sys.path.append('../scripts/')
from wiyot import *

In [4]:
### Read dataset relating a preverb to hierarchy class
preverbhierarchy, preverb2level, preverb2class
preverbhierarchy.head()

,preverb,Class,Level
0,PST,I,1
1,already,I,1
2,EMPH,I,1
3,PROG,I,1
4,then,I,1


### Process Kroeber data

#### Crow story

In [ ]:
## Subset by story
crowStory = getStory(kroeber,'Crow')
## Split into wiyot/glos/translation
crowStory_split = splitStory(crowStory)
print(len(crowStory_split)) ## number of triplets in the spreadsheet
crowStory_split[0]

In [ ]:
### Separate the wiyot/gloss/english
allwiyot,allenglish,fullstory = reformatStory(crowStory_split)

In [ ]:
### Split into words
wiyot_words = story2words(allwiyot)
english_words = story2words(allenglish)
fullstory_words = story2words(fullstory)

In [ ]:
### first two must be the same length
len(english_words), len(wiyot_words), len(fullstory_words)

In [ ]:
### Keep wiyot/gloss together
english2wiyot = list(zip(english_words,wiyot_words))

__assign each word against the hierarchy class__

In [ ]:
english2levelclass = testClass(english_words,preverb2level,preverb2class)
pd.DataFrame(english2levelclass).head(3)

### Test if hierarchy is preserved throughout a sentence

In [ ]:
wiyot_english_class = []
for _ in crowStory_split:
    translation = _[2]
    wiyot,eng = split_words(_[0]), split_words(_[1])
    eng_class = [[preverb2class[_] if preverb2class.get(_) is not None else 0 for _ in word.split('-')] for word in eng]
    eng_class = sum(eng_class,[])
    eng_level = [[preverb2level[_] if preverb2level.get(_) is not None else 0 for _ in word.split('-')] for word in eng]
    eng_level = sum(eng_level,[])
    wiyot_english_class.append([wiyot,eng,eng_class,eng_level])

In [ ]:
story_Wi2Eng_Hierachy = pd.DataFrame(wiyot_english_class,columns=['wiyot','english','class','class_num'])
story_Wi2Eng_Hierachy['result'] = list(map(checkSequenceClass,story_Wi2Eng_Hierachy['class_num']))
story_Wi2Eng_Hierachy.head()

### Extend to other stories

In [5]:
kroeber = pd.read_excel(f'../data/Kroeber texts (2).xlsx',None)
kroeber_results = dict()
for storyname, story in kroeber.items():
    story = getStory(kroeber,storyname)
    tmp = processStory(story) 
    tmp['story_name'] = storyname
    kroeber_results[storyname] = tmp

In [8]:
kroeber_DF = pd.concat(kroeber_results.values())
kroeber_DF['author'] = 'Kroeber'
kroeber_DF.head()

,wiyot,english,class,class_num,result,story_name,author
0,"[Kacid, wule-póyakhd-ił, šudi-lákhow]","[Crow, please-call.kin.in.law-NMLZ.A ocean-o...","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",None,Crow,Kroeber
1,"[wule-póyakhd-ił, hi-wú-lag-ił., Šudi-lákhow]",[please-be.related.by.marriage-NMLZ.A PST-PF...,"[0, 0, 0, II, 0, 0, 0]","[0, 0, 0, 2, 0, 0, 0]",None,Crow,Kroeber
2,"[hi-lès-ił., Cákhi, tu-lùl-úw-ił]",[PST-travel.in.boat-3SBJ.DEF. Child[PL] PL...,"[I, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]",None,Crow,Kroeber
3,"[díkh-ad., Wuphedakáčhkš-ił phlét-w-akw. Kh...",[be.three-3SBJ.STV. Put-3SBJ.DEF rock-SUB-LO...,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",None,Crow,Kroeber
4,"[wuphedakáckš-ił. Hi-khu-lów-ił., Tiwe-dú-lak...",[put-3SBJ.DEF. PST-ITER-come-3SBJ.DEF. What....,"[0, 0, V, 0, 0, 0, 0, 0, 0]","[0, 0, 5, 0, 0, 0, 0, 0, 0]",None,Crow,Kroeber


---

In [37]:
reichards = pd.read_excel(f'../data/Reichard texts (2).xlsx',None)
reichard_results = dict()
for storyname, story in reichards.items():
    try:
        story = getStory(reichards,storyname)
        tmp = processStory(story) 
        tmp['story_name'] = storyname
        reichard_results[storyname] = tmp    
    except:
        print(storyname)

In [39]:
reichards_DF = pd.concat(reichard_results.values())
reichards_DF['author'] = 'Reichards'
reichards_DF.head()

,wiyot,english,class,class_num,result,story_name,author
0,"[Kwétow-ił, carórił kuc-o-ki-łákw-iw-ił.]",[Long.ago-3SBJ.DEF girl be.one-EMP-CESS-gro...,"[0, 0, 0, VI, 0, 0, 0]","[0, 0, 0, 6, 0, 0, 0]",None,Sample Text,Reichards
1,"[Čuwowáł kówa-dár-ił, hawegádak]","[DEM, INCH-do.so-3SBJ.DEF grass]","[0, VII, 0, 0]","[0, 7, 0, 0]",None,Sample Text,Reichards
2,"[ta-lól-ek-w-el, mótih taló-lul-aw-ił.]",[PROG-go.about-NMLZ-SUB-3POSS stick around....,"[I, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0]",None,Sample Text,Reichards
3,"[Motih, talo-tokw-a-phł-óy-aw-eł]","[Stick, around.2-hit-EMP-CL.hairlike-move.asid...","[0, III, 0, 0, 0, 0, 0, 0]","[0, 3, 0, 0, 0, 0, 0, 0]",None,Sample Text,Reichards
4,"[hu-tad-ók-w-ahl, kił khe-wal-ah]",[3POSS-scare-NMLZ.INF-SUB-3POSS CESS PROB-s...,"[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]",None,Sample Text,Reichards


In [40]:
teeter = pd.read_excel(f'../data/Teeter texts (2).xlsx',None)
teeter_results = dict()
for storyname, story in teeter.items():
    try:
        story = getStory(teeter,storyname)
        tmp = processStory(story) 
        tmp['story_name'] = storyname
        teeter_results[storyname] = tmp    
    except:
        print(storyname)

In [41]:
teeter_DF = pd.concat(teeter_results.values())
teeter_DF['author'] = 'Teeter'
teeter_DF.head()

,wiyot,english,class,class_num,result,story_name,author
0,"[Hiwót, kitko kowa šipar-om-á., Kowa]",[Abalone AFF INCH imitate-3OBJ-1SBJ. INCH],"[0, 0, 0]","[0, 0, 0]",None,1. Abalone,Teeter
1,"[lágadahl,, dagadoy-ák-w-ih., Carórił,]",[tell.about.3OBJ[1SBJ] be.so-NMLZ.INF-SUB-NML...,"[0, 0, 0, 0]","[0, 0, 0, 0]",None,1. Abalone,Teeter
2,"[hiwót. Dók,, hišbarí, to-carórił. Dók,]",[abalone. Toward south PROG-young.woman. ...,"[0, 0]","[0, 0]",None,1. Abalone,Teeter
3,"[búhr-ał, thígadił. Diníh., Ki]",[north-ADVS young.man. Dentalium. NEG.EMPH],"[0, 0]","[0, 0]",None,1. Abalone,Teeter
4,"[khúnohnalaw-íł, čawo to lóło<h>w]",[darken-3SBJ.DEF DEM PROG how.look<NMLZ.P...,"[0, 0]","[0, 0]",None,1. Abalone,Teeter


In [44]:
allauthors = pd.concat([teeter_DF,reichards_DF,kroeber_DF])
print(len(allauthors))
allauthors.head()

4390


,wiyot,english,class,class_num,result,story_name,author
0,"[Hiwót, kitko kowa šipar-om-á., Kowa]",[Abalone AFF INCH imitate-3OBJ-1SBJ. INCH],"[0, 0, 0]","[0, 0, 0]",None,1. Abalone,Teeter
1,"[lágadahl,, dagadoy-ák-w-ih., Carórił,]",[tell.about.3OBJ[1SBJ] be.so-NMLZ.INF-SUB-NML...,"[0, 0, 0, 0]","[0, 0, 0, 0]",None,1. Abalone,Teeter
2,"[hiwót. Dók,, hišbarí, to-carórił. Dók,]",[abalone. Toward south PROG-young.woman. ...,"[0, 0]","[0, 0]",None,1. Abalone,Teeter
3,"[búhr-ał, thígadił. Diníh., Ki]",[north-ADVS young.man. Dentalium. NEG.EMPH],"[0, 0]","[0, 0]",None,1. Abalone,Teeter
4,"[khúnohnalaw-íł, čawo to lóło<h>w]",[darken-3SBJ.DEF DEM PROG how.look<NMLZ.P...,"[0, 0]","[0, 0]",None,1. Abalone,Teeter


In [47]:
allauthors.groupby('result').size()

result
False    175
True     564
dtype: int64

In [59]:
564/(175+564)

0.7631935047361299

In [54]:
allauthors.groupby(['author','result']).size().reset_index().to_clipboard()

In [49]:
allauthors[~allauthors['result'].isna()].to_clipboard(index=None)

In [56]:
len(allauthors[allauthors['result'].isna()])/len(allauthors)

0.8316628701594533

In [57]:
allauthors[allauthors['result'].isna()].head()

,wiyot,english,class,class_num,result,story_name,author
0,"[Hiwót, kitko kowa šipar-om-á., Kowa]",[Abalone AFF INCH imitate-3OBJ-1SBJ. INCH],"[0, 0, 0]","[0, 0, 0]",None,1. Abalone,Teeter
1,"[lágadahl,, dagadoy-ák-w-ih., Carórił,]",[tell.about.3OBJ[1SBJ] be.so-NMLZ.INF-SUB-NML...,"[0, 0, 0, 0]","[0, 0, 0, 0]",None,1. Abalone,Teeter
2,"[hiwót. Dók,, hišbarí, to-carórił. Dók,]",[abalone. Toward south PROG-young.woman. ...,"[0, 0]","[0, 0]",None,1. Abalone,Teeter
3,"[búhr-ał, thígadił. Diníh., Ki]",[north-ADVS young.man. Dentalium. NEG.EMPH],"[0, 0]","[0, 0]",None,1. Abalone,Teeter
4,"[khúnohnalaw-íł, čawo to lóło<h>w]",[darken-3SBJ.DEF DEM PROG how.look<NMLZ.P...,"[0, 0]","[0, 0]",None,1. Abalone,Teeter


In [58]:
allauthors[~allauthors['result'].isna()].head()

,wiyot,english,class,class_num,result,story_name,author
18,"[ko-ku-lołól-ih,, khuno ku-to-lołól-ih.]",[NEG-ITER-appear-NMLZ.P before ITER-PROG-ap...,"[II, V, 0, 0, I, 0, 0]","[2, 5, 0, 0, 1, 0, 0]",False,1. Abalone,Teeter
31,"[ko-ku-lołól-ih,, khuno ku to-lołól-ih,]",[NEG-ITER-appear-NMLZ.P before DEM.DIST PRO...,"[II, V, 0, 0, 0, 0]","[2, 5, 0, 0, 0, 0]",True,1. Abalone,Teeter
34,"[to-kiš-ku, tówiłor-ił., To kito]",[PROG-in.vain-ITER look.at-3SBJ.DEF. PROG ...,"[I, IV, 0, 0]","[1, 4, 0, 0]",True,1. Abalone,Teeter
35,"[kos-bú-kot-ak,, kiyo, kut-katolakw to k...",[then.2-PFV-stay-NMLZ.INF DUB one-month ...,"[IV, II, 0, 0, 0]","[4, 2, 0, 0, 0]",False,1. Abalone,Teeter
47,"[kho-ku lág-ih. Hi yíl-ił, ła-pič]",[NEG-ITER go-NEG. PST say-3SBJ.DEF alrea...,"[II, 0, 0, 0, IX]","[2, 0, 0, 0, 9]",True,1. Abalone,Teeter


In [ ]:
 564/175